In [1]:
# --- 1. Setup Environment ---
!pip uninstall -y jax jaxlib tensorflow flax -q
!pip install -U transformers datasets evaluate accelerate codecarbon -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 104.6 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 42.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:

In [2]:
# --- 2. Imports ---
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import torch
from codecarbon import EmissionsTracker


In [3]:
# --- 3. Load Dataset (IMDB for Sentiment Classification) ---
dataset = load_dataset("imdb")


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# --- 4. Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# --- 5. Train/Test Split ---
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))


In [7]:
# --- 6. Define Model ---
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# --- 7. Define Metric (Accuracy) ---
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [12]:
# --- 8. TrainingArguments ---
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"  # disables wandb etc.
)

In [13]:
# --- 9. Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipykernel_36/2457266571.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# --- 10. Track Energy with CodeCarbon ---
tracker = EmissionsTracker(output_dir="./", output_file="emissions.csv")
tracker.start()

trainer.train()
results = trainer.evaluate()

tracker.stop()


[codecarbon WARNING @ 06:05:06] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 06:05:06] [setup] RAM Tracking...
[codecarbon INFO @ 06:05:06] [setup] CPU Tracking...
[codecarbon WARNING @ 06:05:07] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 06:05:07] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 06:05:07] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 06:05:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 06:05:07] [setup] GPU Tracking...
[codecarbon INFO @ 06:05:07] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 06:05:07] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

Epoch,Training Loss,Validation Loss,Accuracy
1,0.317700,0.295303,0.875000
2,0.218200,0.273474,0.883500


[codecarbon INFO @ 06:05:25] Energy consumed for RAM : 0.000083 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 06:05:25] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 06:05:25] Energy consumed for All CPU : 0.000177 kWh
[codecarbon INFO @ 06:05:25] Energy consumed for all GPUs : 0.000473 kWh. Total GPU Power : 113.35770669751145 W
[codecarbon INFO @ 06:05:25] 0.000733 kWh of electricity used since the beginning.
[codecarbon INFO @ 06:05:40] Energy consumed for RAM : 0.000167 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 06:05:40] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 06:05:40] Energy consumed for All CPU : 0.000354 kWh
[codecarbon INFO @ 06:05:40] Energy consumed for all GPUs : 0.001013 kWh. Total GPU Power : 129.7639519324354 W
[codecarbon INFO @ 06:05:40] 0.001534 kWh of electricity used since the beginning.
[codecarbon INFO @ 06:05:55] Energy consumed for RAM : 0.000250 kWh. RAM Power : 

[codecarbon INFO @ 06:07:40] Energy consumed for RAM : 0.000833 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 06:07:40] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 06:07:40] Energy consumed for All CPU : 0.001770 kWh
[codecarbon INFO @ 06:07:40] Energy consumed for all GPUs : 0.005338 kWh. Total GPU Power : 130.26881948935628 W
[codecarbon INFO @ 06:07:40] 0.007941 kWh of electricity used since the beginning.
[codecarbon INFO @ 06:07:44] Energy consumed for RAM : 0.000854 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 06:07:44] Delta energy consumed for CPU with constant : 0.000044 kWh, power : 42.5 W
[codecarbon INFO @ 06:07:44] Energy consumed for All CPU : 0.001814 kWh
[codecarbon INFO @ 06:07:44] Energy consumed for all GPUs : 0.005474 kWh. Total GPU Power : 129.9605907413675 W
[codecarbon INFO @ 06:07:44] 0.008142 kWh of electricity used since the beginning.


0.0036852646073084047

In [15]:
# --- 11. Show Results ---
print("Final Accuracy:", results["eval_accuracy"])
print("CodeCarbon log saved to emissions.csv")

Final Accuracy: 0.8835
CodeCarbon log saved to emissions.csv
